# Mpls Marriott Hotel Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.
4. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion
5. Rerun batch sims in EnergyPlus v9.5.
6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.
    - scenario (1-exist, 2-clear panel, 3-lowe panel)
    - Elevation (e-project 34, sw-project 36, nw-project37)
    - GlazingSystem (1-exist, 2-clear panel, 3-lowe panel)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
comgas_cost = 0.60
comelec_cost = 0.1043

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.65

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MMBtu) 323.34 (lb/kbtu).32334 ####
#Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####

gas_co2 = 0.14566
elec_co2 = 0.2964

In [5]:
# Zone size for EUI and cost/sf

sf = 210

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory Commercial Gas Heating

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/msp/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/msp/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
# all_ComElec

# 394200  rows, 44 columns

In [10]:
# list(all_ComElec.columns.values)

In [11]:
# Get wanted columns
all_ComElec1 = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

# all_ComElec1

In [12]:
all_ComElec1 = all_ComElec1.dropna().reset_index(drop=True)

#all_ComElec1

In [13]:
# # Export grouped file to csv
# all_ComElec1.to_csv("Scraper_Output/msp_all.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [72]:
# Rename Columns
all_ComElec2 = all_ComElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

all_ComElec2

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly)
0,1Eexist,01/01 01:00:00,E,Exist,0,1691706.767,1.389808e+06,0.0,39312
1,1Eexist,01/01 02:00:00,E,Exist,0,1980664.967,1.389808e+06,0.0,39312
2,1Eexist,01/01 03:00:00,E,Exist,0,2001692.190,1.389808e+06,0.0,16632
3,1Eexist,01/01 04:00:00,E,Exist,0,1983465.003,1.389808e+06,0.0,16632
4,1Eexist,01/01 05:00:00,E,Exist,0,1958731.477,1.389808e+06,0.0,16632
...,...,...,...,...,...,...,...,...,...
78835,3NWlowe,12/31 20:00:00,NW,LowE Panel,0,1835072.005,5.281277e+05,0.0,151200
78836,3NWlowe,12/31 21:00:00,NW,LowE Panel,0,1705080.404,5.281277e+05,0.0,151200
78837,3NWlowe,12/31 22:00:00,NW,LowE Panel,0,1734783.158,5.281277e+05,0.0,151200
78838,3NWlowe,12/31 23:00:00,NW,LowE Panel,0,2030896.714,5.281277e+05,0.0,128520


In [73]:
list(all_ComElec2.columns.values)

['Scenario',
 'Date_Time',
 'Elevation',
 'GlazingSystem',
 'HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)']

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [15]:
# Copy dataframe to modify and leave original df intact
get_annual_ComElec = all_ComElec2.copy(deep=True)

#get_annual_ComElec

In [16]:
# Convert Joules to KBtu in dataframe
get_annual_ComElec["FanEnergy(kBtu)"] = get_annual_ComElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingElecEnergy(kBtu)"] = get_annual_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingGasEnergy(kBtu)"] = get_annual_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["CoolingEnergy(kBtu)"] = get_annual_ComElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ComElec["LightingEnergy(kBtu)"] = (get_annual_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_ComElec

In [17]:
# Drop columns
get_annual_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ComElec

In [18]:
# Add energy totals
totalComEnergy = get_annual_ComElec["FanEnergy(kBtu)"] + get_annual_ComElec["HeatingElecEnergy(kBtu)"] + get_annual_ComElec["LightingEnergy(kBtu)"] + get_annual_ComElec["CoolingEnergy(kBtu)"]
        
get_annual_ComElec["AnnualEnergy(kBtu)"] = totalComEnergy

get_annual_ComElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu)
0,1Eexist,01/01 01:00:00,E,Exist,1.317283,0.0,1.603429,0.0,0.037261,1.354544
1,1Eexist,01/01 02:00:00,E,Exist,1.317283,0.0,1.877308,0.0,0.037261,1.354544
2,1Eexist,01/01 03:00:00,E,Exist,1.317283,0.0,1.897238,0.0,0.015764,1.333047
3,1Eexist,01/01 04:00:00,E,Exist,1.317283,0.0,1.879962,0.0,0.015764,1.333047
4,1Eexist,01/01 05:00:00,E,Exist,1.317283,0.0,1.856519,0.0,0.015764,1.333047
...,...,...,...,...,...,...,...,...,...,...
78835,3NWlowe,12/31 20:00:00,NW,LowE Panel,0.500568,0.0,1.739313,0.0,0.143310,0.643878
78836,3NWlowe,12/31 21:00:00,NW,LowE Panel,0.500568,0.0,1.616104,0.0,0.143310,0.643878
78837,3NWlowe,12/31 22:00:00,NW,LowE Panel,0.500568,0.0,1.644257,0.0,0.143310,0.643878
78838,3NWlowe,12/31 23:00:00,NW,LowE Panel,0.500568,0.0,1.924919,0.0,0.121813,0.622382


-----

## Get Energy per sf 
Add EUI (site) column<br>


In [19]:
# Get data per sf (energy/sf)
get_annual_ComElec["FanEnergy(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ComElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ComElec["LightingEnergy(kBtu/sf)"] = (get_annual_ComElec["LightingEnergy(kBtu)"]) / sf

In [20]:
# Establish EUI (energy/sf)
get_annual_ComElec["EUI(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingGasEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +  
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
                                                       
get_annual_ComElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),LightingEnergy(kBtu/sf),EUI(kBtu/sf)
0,1Eexist,01/01 01:00:00,E,Exist,1.317283,0.0,1.603429,0.0,0.037261,1.354544,0.006273,0.0,0.007635,0.0,0.000177,0.014086
1,1Eexist,01/01 02:00:00,E,Exist,1.317283,0.0,1.877308,0.0,0.037261,1.354544,0.006273,0.0,0.008940,0.0,0.000177,0.015390
2,1Eexist,01/01 03:00:00,E,Exist,1.317283,0.0,1.897238,0.0,0.015764,1.333047,0.006273,0.0,0.009034,0.0,0.000075,0.015382
3,1Eexist,01/01 04:00:00,E,Exist,1.317283,0.0,1.879962,0.0,0.015764,1.333047,0.006273,0.0,0.008952,0.0,0.000075,0.015300
4,1Eexist,01/01 05:00:00,E,Exist,1.317283,0.0,1.856519,0.0,0.015764,1.333047,0.006273,0.0,0.008841,0.0,0.000075,0.015188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78835,3NWlowe,12/31 20:00:00,NW,LowE Panel,0.500568,0.0,1.739313,0.0,0.143310,0.643878,0.002384,0.0,0.008282,0.0,0.000682,0.011349
78836,3NWlowe,12/31 21:00:00,NW,LowE Panel,0.500568,0.0,1.616104,0.0,0.143310,0.643878,0.002384,0.0,0.007696,0.0,0.000682,0.010762
78837,3NWlowe,12/31 22:00:00,NW,LowE Panel,0.500568,0.0,1.644257,0.0,0.143310,0.643878,0.002384,0.0,0.007830,0.0,0.000682,0.010896
78838,3NWlowe,12/31 23:00:00,NW,LowE Panel,0.500568,0.0,1.924919,0.0,0.121813,0.622382,0.002384,0.0,0.009166,0.0,0.000580,0.012130


-----

## Get Carbon per sf 
Add carbon/sf column

In [21]:
# Do carbon calculation using variable (lb/sf)
get_annual_ComElec["CO2_Electric(lb/sf)"] = ((get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ComElec

In [22]:
get_annual_ComElec["CO2_Gas(lb/sf)"] = ((get_annual_ComElec["HeatingGasEnergy(kBtu)"]) * gas_co2) / sf
                                                       
get_annual_ComElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf),CO2_Gas(lb/sf)
0,1Eexist,01/01 01:00:00,E,Exist,1.317283,0.0,1.603429,0.0,0.037261,1.354544,0.006273,0.0,0.007635,0.0,0.000177,0.014086,0.001912,0.001112
1,1Eexist,01/01 02:00:00,E,Exist,1.317283,0.0,1.877308,0.0,0.037261,1.354544,0.006273,0.0,0.008940,0.0,0.000177,0.015390,0.001912,0.001302
2,1Eexist,01/01 03:00:00,E,Exist,1.317283,0.0,1.897238,0.0,0.015764,1.333047,0.006273,0.0,0.009034,0.0,0.000075,0.015382,0.001882,0.001316
3,1Eexist,01/01 04:00:00,E,Exist,1.317283,0.0,1.879962,0.0,0.015764,1.333047,0.006273,0.0,0.008952,0.0,0.000075,0.015300,0.001882,0.001304
4,1Eexist,01/01 05:00:00,E,Exist,1.317283,0.0,1.856519,0.0,0.015764,1.333047,0.006273,0.0,0.008841,0.0,0.000075,0.015188,0.001882,0.001288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78835,3NWlowe,12/31 20:00:00,NW,LowE Panel,0.500568,0.0,1.739313,0.0,0.143310,0.643878,0.002384,0.0,0.008282,0.0,0.000682,0.011349,0.000909,0.001206
78836,3NWlowe,12/31 21:00:00,NW,LowE Panel,0.500568,0.0,1.616104,0.0,0.143310,0.643878,0.002384,0.0,0.007696,0.0,0.000682,0.010762,0.000909,0.001121
78837,3NWlowe,12/31 22:00:00,NW,LowE Panel,0.500568,0.0,1.644257,0.0,0.143310,0.643878,0.002384,0.0,0.007830,0.0,0.000682,0.010896,0.000909,0.001140
78838,3NWlowe,12/31 23:00:00,NW,LowE Panel,0.500568,0.0,1.924919,0.0,0.121813,0.622382,0.002384,0.0,0.009166,0.0,0.000580,0.012130,0.000878,0.001335


## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [23]:
# Get annual costs
get_annual_costs_ComElec = all_ComElec2.copy(deep=True)

In [31]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ComElec["FanCost($)"] = (get_annual_costs_ComElec["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs_ComElec["HeatingElecCost($)"] = (get_annual_costs_ComElec["HeatingElecEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs_ComElec["HeatingGasCost($)"] = ((get_annual_costs_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["CoolingCost($)"] = (get_annual_costs_ComElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ComElec["LightingCost($)"] = (get_annual_costs_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux * comelec_cost

#get_annual_costs

In [32]:
# Drop columns
get_annual_costs_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs_ComElec

In [33]:
# Add cost totals
totalComCost = get_annual_costs_ComElec["FanCost($)"] + get_annual_costs_ComElec["HeatingElecCost($)"] + get_annual_costs_ComElec["HeatingGasCost($)"]  + get_annual_costs_ComElec["CoolingCost($)"] + get_annual_costs_ComElec["LightingCost($)"]
        
get_annual_costs_ComElec["AnnualCost($)"] = totalComCost

# get_annual_costs_ComElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>


In [34]:
# Get data per sf ($/sf)
get_annual_costs_ComElec["FanCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"]) / sf
get_annual_costs_ComElec["HeatingElecCost($/sf)"] = (get_annual_costs_ComElec["HeatingElecCost($)"]) / sf
get_annual_costs_ComElec["HeatingGasCost($/sf)"] = (get_annual_costs_ComElec["HeatingGasCost($)"]) / sf
get_annual_costs_ComElec["CoolingCost($/sf)"] = (get_annual_costs_ComElec["CoolingCost($)"]) / sf
get_annual_costs_ComElec["LightingCost($/sf)"] = (get_annual_costs_ComElec["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [35]:
# Establish costs ($/sf)
get_annual_costs_ComElec["AnnualCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"] +
                                    get_annual_costs_ComElec["HeatingElecCost($)"] +
                                    get_annual_costs_ComElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ComElec["CoolingCost($)"] +  
                                    get_annual_costs_ComElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ComElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [36]:
# # Combine data for energy and cost
allAnnualDataComElec = pd.merge(get_annual_ComElec, get_annual_costs_ComElec)

allAnnualDataComElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),...,HeatingGasCost($),CoolingCost($),LightingCost($),AnnualCost($),FanCost($/sf),HeatingElecCost($/sf),HeatingGasCost($/sf),CoolingCost($/sf),LightingCost($/sf),AnnualCost($/sf)
0,1Eexist,01/01 01:00:00,E,Exist,1.317283,0.0,1.603429,0.0,0.037261,1.354544,...,0.009621,0.0,0.003886,0.053773,0.000192,0.0,0.000046,0.0,0.000019,0.000256
1,1Eexist,01/01 02:00:00,E,Exist,1.317283,0.0,1.877308,0.0,0.037261,1.354544,...,0.011264,0.0,0.003886,0.055416,0.000192,0.0,0.000054,0.0,0.000019,0.000264
2,1Eexist,01/01 03:00:00,E,Exist,1.317283,0.0,1.897238,0.0,0.015764,1.333047,...,0.011383,0.0,0.001644,0.053293,0.000192,0.0,0.000054,0.0,0.000008,0.000254
3,1Eexist,01/01 04:00:00,E,Exist,1.317283,0.0,1.879962,0.0,0.015764,1.333047,...,0.011280,0.0,0.001644,0.053190,0.000192,0.0,0.000054,0.0,0.000008,0.000253
4,1Eexist,01/01 05:00:00,E,Exist,1.317283,0.0,1.856519,0.0,0.015764,1.333047,...,0.011139,0.0,0.001644,0.053049,0.000192,0.0,0.000053,0.0,0.000008,0.000253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78835,3NWlowe,12/31 20:00:00,NW,LowE Panel,0.500568,0.0,1.739313,0.0,0.143310,0.643878,...,0.010436,0.0,0.014947,0.040684,0.000073,0.0,0.000050,0.0,0.000071,0.000194
78836,3NWlowe,12/31 21:00:00,NW,LowE Panel,0.500568,0.0,1.616104,0.0,0.143310,0.643878,...,0.009697,0.0,0.014947,0.039945,0.000073,0.0,0.000046,0.0,0.000071,0.000190
78837,3NWlowe,12/31 22:00:00,NW,LowE Panel,0.500568,0.0,1.644257,0.0,0.143310,0.643878,...,0.009866,0.0,0.014947,0.040114,0.000073,0.0,0.000047,0.0,0.000071,0.000191
78838,3NWlowe,12/31 23:00:00,NW,LowE Panel,0.500568,0.0,1.924919,0.0,0.121813,0.622382,...,0.011550,0.0,0.012705,0.039556,0.000073,0.0,0.000055,0.0,0.000061,0.000188


In [37]:
# list(allAnnualDataComElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [38]:
# Group by scenario
annualDataCom = allAnnualDataComElec.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualDataCom.sum()

In [39]:
annualDataComFinal = annualDataCom.sum()

annualDataComFinal

,,,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),AnnualEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),...,HeatingGasCost($),CoolingCost($),LightingCost($),AnnualCost($),FanCost($/sf),HeatingElecCost($/sf),HeatingGasCost($/sf),CoolingCost($/sf),LightingCost($/sf),AnnualCost($/sf)
Scenario,Elevation,GlazingSystem,,,,,,,,,,,,,,,,,,,,,
1Eexist,E,Exist,11539.402491,0.0,3047.905108,6128.168118,517.381876,18184.952485,54.949536,0.0,14.513834,29.181753,...,18.287431,187.321631,53.96293,612.300515,1.679660,0.0,0.087083,0.892008,0.256966,2.915717
1NWexist,NW,Exist,5531.677939,0.0,10939.317555,2770.664011,517.381876,8819.723825,26.341324,0.0,52.091988,13.193638,...,65.635905,84.691753,53.96293,373.379117,0.805183,0.0,0.312552,0.403294,0.256966,1.777996
1SWexist,SW,Exist,7919.625871,0.0,5063.978717,5647.202682,517.381876,14084.210429,37.712504,0.0,24.114184,26.891441,...,30.383872,172.619810,53.96293,499.048292,1.152770,0.0,0.144685,0.821999,0.256966,2.376420
2Eclear,E,Clear Panel,10346.244669,0.0,1632.206900,5891.609641,517.381876,16755.236186,49.267832,0.0,7.772414,28.055284,...,9.793241,180.090674,53.96293,560.103741,1.505985,0.0,0.046634,0.857575,0.256966,2.667161
2NWclear,NW,Clear Panel,5003.128528,0.0,8375.998724,2639.493292,517.381876,8160.003695,23.824422,0.0,39.885708,12.569016,...,50.255992,80.682217,53.96293,337.833333,0.728249,0.0,0.239314,0.384201,0.256966,1.608730
2SWclear,SW,Clear Panel,6976.354481,0.0,3564.273408,5231.323035,517.381876,12725.059391,33.220736,0.0,16.972731,24.911062,...,21.385640,159.907487,53.96293,448.504465,1.015469,0.0,0.101836,0.761464,0.256966,2.135736
3Elowe,E,LowE Panel,8991.220597,0.0,2538.270256,4844.037243,517.381876,14352.639715,42.815336,0.0,12.087001,23.066844,...,15.229622,148.069201,53.96293,492.099203,1.308750,0.0,0.072522,0.705091,0.256966,2.343330
3NWlowe,NW,LowE Panel,4384.980011,0.0,9497.675181,2249.446595,517.381876,7151.808481,20.880857,0.0,45.227025,10.711650,...,56.986051,68.759537,53.96293,313.745573,0.638272,0.0,0.271362,0.327426,0.256966,1.494027
3SWlowe,SW,LowE Panel,6146.367874,0.0,4620.680189,4240.850881,517.381876,10904.600630,29.268418,0.0,22.003239,20.194528,...,27.724081,129.631415,53.96293,399.196374,0.894657,0.0,0.132019,0.617292,0.256966,1.900935


In [40]:
list(annualDataComFinal.columns.values)

['FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)',
 'FanCost($)',
 'HeatingElecCost($)',
 'HeatingGasCost($)',
 'CoolingCost($)',
 'LightingCost($)',
 'AnnualCost($)',
 'FanCost($/sf)',
 'HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)',
 'CoolingCost($/sf)',
 'LightingCost($/sf)',
 'AnnualCost($/sf)']

In [41]:
# Export grouped file to csv
# annualDataComFinal.to_csv("Scraper_Output/msp_annual_com.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Export to CSV files


In [42]:
# Define the scenario for the existing condition.
baselineAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Exist"]

# baselineAnnualDataComElec

In [43]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualDataComElec = baselineAnnualDataComElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'B_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'B_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'B_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'B_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingElecCost($)':'B_HeatingElecCost($)',
 'HeatingGasCost($)':'B_HeatingGasCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingElecCost($/sf)':'B_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'B_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

# allBaselineAnnualDataComElec

In [44]:
# Reset index
allBaselineAnnualDataComElec.reset_index(inplace=True, drop=True)

# allBaselineAnnualDataComElec

In [45]:
# # Define the scenario for the baselines.

clearAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Clear Panel"]

# clearAnnualDataComElec

In [46]:
# Rename columns to indicate baseline to prepare for merge"

allClearAnnualDataComElec = clearAnnualDataComElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'FanEnergy(kBtu)':'C_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'C_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'C_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'C_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'C_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'C_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'C_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'C_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'C_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'C_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'C_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'C_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'C_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'C_CO2_Gas(lb/sf)',
 'FanCost($)':'C_FanCost($)',
 'HeatingElecCost($)':'C_HeatingElecCost($)',
 'HeatingGasCost($)':'C_HeatingGasCost($)',
 'CoolingCost($)':'C_CoolingCost($)',
 'LightingCost($)':'C_LightingCost($)',
 'AnnualCost($)':'C_AnnualCost($)',
 'FanCost($/sf)':'C_FanCost($/sf)',
 'HeatingElecCost($/sf)':'C_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'C_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'C_CoolingCost($/sf)',
 'LightingCost($/sf)':'C_LightingCost($/sf)',
 'AnnualCost($/sf)':'C_AnnualCost($/sf)'
                            })

# allClearAnnualDataComElec

In [47]:
# Reset index
allClearAnnualDataComElec.reset_index(inplace=True, drop=True)

# allClearAnnualDataComElec

In [48]:
# Define the scenario for the baselines.

loweAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "LowE Panel"]

# loweAnnualDataComElec

In [49]:
# Rename columns to indicate baseline to prepare for merge"

allLoweAnnualDataComElec = loweAnnualDataComElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'FanEnergy(kBtu)':'L_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'L_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'L_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'L_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'L_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'L_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'L_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'L_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'L_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'L_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'L_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'L_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'L_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'L_CO2_Gas(lb/sf)',
 'FanCost($)':'L_FanCost($)',
 'HeatingElecCost($)':'L_HeatingElecCost($)',
 'HeatingGasCost($)':'L_HeatingGasCost($)',
 'CoolingCost($)':'L_CoolingCost($)',
 'LightingCost($)':'L_LightingCost($)',
 'AnnualCost($)':'L_AnnualCost($)',
 'FanCost($/sf)':'L_FanCost($/sf)',
 'HeatingElecCost($/sf)':'L_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'L_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'L_CoolingCost($/sf)',
 'LightingCost($/sf)':'L_LightingCost($/sf)',
 'AnnualCost($/sf)':'L_AnnualCost($/sf)'
                            })

# allLoweAnnualDataComElec

In [50]:
#Reset index
allLoweAnnualDataComElec.reset_index(inplace=True, drop=True)

# allLoweAnnualDataComElec

In [51]:
# # Export individual CSV files for each condition

# allBaselineAnnualDataComElec.to_csv("Scraper_Output/msp_exist_all_annual_com.csv", header=True, index=True)
# allClearAnnualDataComElec.to_csv("Scraper_Output/msp_clear_all_annual_com.csv", header=True, index=True)
# allLoweAnnualDataComElec.to_csv("Scraper_Output/msp_lowe_all_annual_com.csv", header=True, index=True)

---
## Merge the 3 dataframes to perform delta from existing calculation


In [52]:
# merge existing and clear
MergedAnnualData = pd.merge(allBaselineAnnualDataComElec, allClearAnnualDataComElec, left_index=True, right_index=True)

# merge the above with lowe
AllMergedAnnualData = pd.merge(MergedAnnualData, allLoweAnnualDataComElec, left_index=True, right_index=True)

# AllMergedAnnualData

In [53]:
# list(AllMergedAnnualData.columns.values)

In [54]:
# Deltas of exist - clear

AllMergedAnnualData["DeltaClear_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["C_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingElecEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingElecEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingElecEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingGasEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingGasEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingGasEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["C_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["C_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["C_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaClear_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["C_FanCost($)"]
AllMergedAnnualData["DeltaClear_HeatingElecCost($)"] = AllMergedAnnualData["B_HeatingElecCost($)"] - AllMergedAnnualData["C_HeatingElecCost($)"]
AllMergedAnnualData["DeltaClear_HeatingGasCost($)"] = AllMergedAnnualData["B_HeatingGasCost($)"] - AllMergedAnnualData["C_HeatingGasCost($)"]
AllMergedAnnualData["DeltaClear_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["C_CoolingCost($)"]
AllMergedAnnualData["DeltaClear_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["C_LightingCost($)"]
AllMergedAnnualData["DeltaClear_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["C_AnnualCost($)"]
AllMergedAnnualData["DeltaClear_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["C_FanCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingElecCost($/sf)"] = AllMergedAnnualData["B_HeatingElecCost($/sf)"] - AllMergedAnnualData["C_HeatingElecCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingGasCost($/sf)"] = AllMergedAnnualData["B_HeatingGasCost($/sf)"] - AllMergedAnnualData["C_HeatingGasCost($/sf)"]
AllMergedAnnualData["DeltaClear_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["C_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaClear_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["C_LightingCost($/sf)"]
AllMergedAnnualData["DeltaClear_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["C_AnnualCost($/sf)"]


In [55]:
# list(AllMergedAnnualData.columns.values)

In [56]:
# Deltas of exist - lowe

AllMergedAnnualData["DeltaLowe_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["L_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingElecEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingElecEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingElecEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingGasEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingGasEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingGasEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["L_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["L_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["L_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaLowe_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["L_FanCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingElecCost($)"] = AllMergedAnnualData["B_HeatingElecCost($)"] - AllMergedAnnualData["L_HeatingElecCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingGasCost($)"] = AllMergedAnnualData["B_HeatingGasCost($)"] - AllMergedAnnualData["L_HeatingGasCost($)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["L_CoolingCost($)"]
AllMergedAnnualData["DeltaLowe_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["L_LightingCost($)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["L_AnnualCost($)"]
AllMergedAnnualData["DeltaLowe_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["L_FanCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingElecCost($/sf)"] = AllMergedAnnualData["B_HeatingElecCost($/sf)"] - AllMergedAnnualData["L_HeatingElecCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingGasCost($/sf)"] = AllMergedAnnualData["B_HeatingGasCost($/sf)"] - AllMergedAnnualData["L_HeatingGasCost($/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["L_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["L_LightingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["L_AnnualCost($/sf)"]


In [57]:
# list(AllMergedAnnualData.columns.values)

In [58]:
AllMergedAnnualData

,B_Scenario,B_Date_Time,B_Elevation,B_GlazingSystem,B_FanEnergy(kBtu),B_HeatingElecEnergy(kBtu),B_HeatingGasEnergy(kBtu),B_CoolingEnergy(kBtu),B_LightingEnergy(kBtu),B_AnnualEnergy(kBtu),...,DeltaLowe_HeatingGasCost($),DeltaLowe_CoolingCost($),DeltaLowe_LightingCost($),DeltaLowe_AnnualCost($),DeltaLowe_FanCost($/sf),DeltaLowe_HeatingElecCost($/sf),DeltaLowe_HeatingGasCost($/sf),DeltaLowe_CoolingCost($/sf),DeltaLowe_LightingCost($/sf),DeltaLowe_AnnualCost($/sf)
0,1Eexist,01/01 01:00:00,E,Exist,1.317283,0.0,1.603429,0.0,0.037261,1.354544,...,0.001636,0.0,0.0,0.010528,0.000042,0.0,0.000008,0.0,0.0,0.000050
1,1Eexist,01/01 02:00:00,E,Exist,1.317283,0.0,1.877308,0.0,0.037261,1.354544,...,0.001970,0.0,0.0,0.010861,0.000042,0.0,0.000009,0.0,0.0,0.000052
2,1Eexist,01/01 03:00:00,E,Exist,1.317283,0.0,1.897238,0.0,0.015764,1.333047,...,0.002026,0.0,0.0,0.010917,0.000042,0.0,0.000010,0.0,0.0,0.000052
3,1Eexist,01/01 04:00:00,E,Exist,1.317283,0.0,1.879962,0.0,0.015764,1.333047,...,0.002028,0.0,0.0,0.010919,0.000042,0.0,0.000010,0.0,0.0,0.000052
4,1Eexist,01/01 05:00:00,E,Exist,1.317283,0.0,1.856519,0.0,0.015764,1.333047,...,0.002028,0.0,0.0,0.010919,0.000042,0.0,0.000010,0.0,0.0,0.000052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26275,1NWexist,12/31 20:00:00,NW,Exist,0.631470,0.0,2.064089,0.0,0.143310,0.774780,...,0.001949,0.0,0.0,0.005950,0.000019,0.0,0.000009,0.0,0.0,0.000028
26276,1NWexist,12/31 21:00:00,NW,Exist,0.631470,0.0,1.947006,0.0,0.143310,0.774780,...,0.001985,0.0,0.0,0.005987,0.000019,0.0,0.000009,0.0,0.0,0.000029
26277,1NWexist,12/31 22:00:00,NW,Exist,0.631470,0.0,1.985043,0.0,0.143310,0.774780,...,0.002045,0.0,0.0,0.006046,0.000019,0.0,0.000010,0.0,0.0,0.000029
26278,1NWexist,12/31 23:00:00,NW,Exist,0.631470,0.0,2.281612,0.0,0.121813,0.753284,...,0.002140,0.0,0.0,0.006141,0.000019,0.0,0.000010,0.0,0.0,0.000029


---
## Group and Export Annual Delta Data
Group dataframe by scenario<br>
Export to file

In [59]:
# Group by scenario
AnnualDataDelta = AllMergedAnnualData.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=False)

AnnualDataDelta.sum()

,B_Scenario,B_Elevation,B_GlazingSystem,B_FanEnergy(kBtu),B_HeatingElecEnergy(kBtu),B_HeatingGasEnergy(kBtu),B_CoolingEnergy(kBtu),B_LightingEnergy(kBtu),B_AnnualEnergy(kBtu),B_FanEnergy(kBtu/sf),...,DeltaLowe_HeatingGasCost($),DeltaLowe_CoolingCost($),DeltaLowe_LightingCost($),DeltaLowe_AnnualCost($),DeltaLowe_FanCost($/sf),DeltaLowe_HeatingElecCost($/sf),DeltaLowe_HeatingGasCost($/sf),DeltaLowe_CoolingCost($/sf),DeltaLowe_LightingCost($/sf),DeltaLowe_AnnualCost($/sf)
0,1Eexist,E,Exist,11539.402491,0.0,3047.905108,6128.168118,517.381876,18184.952485,54.949536,...,3.057809,39.252430,0.0,120.201312,0.370910,0.0,0.014561,0.186916,0.0,0.572387
1,1NWexist,NW,Exist,5531.677939,0.0,10939.317555,2770.664011,517.381876,8819.723825,26.341324,...,8.649854,15.932216,0.0,59.633544,0.166912,0.0,0.041190,0.075868,0.0,0.283969
2,1SWexist,SW,Exist,7919.625871,0.0,5063.978717,5647.202682,517.381876,14084.210429,37.712504,...,2.659791,42.988395,0.0,99.851918,0.258113,0.0,0.012666,0.204707,0.0,0.475485


In [60]:
list(AnnualDataDelta.sum().columns.values)

['B_Scenario',
 'B_Elevation',
 'B_GlazingSystem',
 'B_FanEnergy(kBtu)',
 'B_HeatingElecEnergy(kBtu)',
 'B_HeatingGasEnergy(kBtu)',
 'B_CoolingEnergy(kBtu)',
 'B_LightingEnergy(kBtu)',
 'B_AnnualEnergy(kBtu)',
 'B_FanEnergy(kBtu/sf)',
 'B_HeatingElecEnergy(kBtu/sf)',
 'B_HeatingGasEnergy(kBtu/sf)',
 'B_CoolingEnergy(kBtu/sf)',
 'B_LightingEnergy(kBtu/sf)',
 'B_EUI(kBtu/sf)',
 'B_CO2_Electric(lb/sf)',
 'B_CO2_Gas(lb/sf)',
 'B_FanCost($)',
 'B_HeatingElecCost($)',
 'B_HeatingGasCost($)',
 'B_CoolingCost($)',
 'B_LightingCost($)',
 'B_AnnualCost($)',
 'B_FanCost($/sf)',
 'B_HeatingElecCost($/sf)',
 'B_HeatingGasCost($/sf)',
 'B_CoolingCost($/sf)',
 'B_LightingCost($/sf)',
 'B_AnnualCost($/sf)',
 'C_FanEnergy(kBtu)',
 'C_HeatingElecEnergy(kBtu)',
 'C_HeatingGasEnergy(kBtu)',
 'C_CoolingEnergy(kBtu)',
 'C_LightingEnergy(kBtu)',
 'C_AnnualEnergy(kBtu)',
 'C_FanEnergy(kBtu/sf)',
 'C_HeatingElecEnergy(kBtu/sf)',
 'C_HeatingGasEnergy(kBtu/sf)',
 'C_CoolingEnergy(kBtu/sf)',
 'C_LightingEnergy(kB

In [61]:
AnnualDataDelta = AnnualDataDelta.sum()

In [62]:
# Export grouped file to csv
AnnualDataDelta.to_csv("Scraper_Output/msp_deltas_annual_com.csv", header=True, index=False)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [68]:
# Copy dataframe to modify and leave original df intact
get_peak_ComElec = all_ComElec2.copy(deep=True)

get_peak_ComElec

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly)
0,1Eexist,01/01 01:00:00,E,Exist,0,1691706.767,1.389808e+06,0.0,39312
1,1Eexist,01/01 02:00:00,E,Exist,0,1980664.967,1.389808e+06,0.0,39312
2,1Eexist,01/01 03:00:00,E,Exist,0,2001692.190,1.389808e+06,0.0,16632
3,1Eexist,01/01 04:00:00,E,Exist,0,1983465.003,1.389808e+06,0.0,16632
4,1Eexist,01/01 05:00:00,E,Exist,0,1958731.477,1.389808e+06,0.0,16632
...,...,...,...,...,...,...,...,...,...
78835,3NWlowe,12/31 20:00:00,NW,LowE Panel,0,1835072.005,5.281277e+05,0.0,151200
78836,3NWlowe,12/31 21:00:00,NW,LowE Panel,0,1705080.404,5.281277e+05,0.0,151200
78837,3NWlowe,12/31 22:00:00,NW,LowE Panel,0,1734783.158,5.281277e+05,0.0,151200
78838,3NWlowe,12/31 23:00:00,NW,LowE Panel,0,2030896.714,5.281277e+05,0.0,128520


In [69]:
list(get_peak_ComElec.columns.values)

['Scenario',
 'Date_Time',
 'Elevation',
 'GlazingSystem',
 'HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)']

In [64]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalComPeak = (get_peak_ComElec["FanEnergy[J](Hourly)"] + get_peak_ComElec["HeatingElecEnergy[J](Hourly)"] + get_peak_ComElec["HeatingGasEnergy[J](Hourly)"] + get_peak_ComElec["CoolingEnergy[J](Hourly)"] + get_peak_ComElec["LightingEnergy[J](Hourly)"])
        
get_peak_ComElec["PeakEnergy[J](Hourly)"] = totalComPeak

get_peak_ComElec["PeakEnergy(w)"] = (get_peak_ComElec["PeakEnergy[J](Hourly)"] / 3600)

get_peak_ComElec["PeakEnergy(w/sf)"] = (get_peak_ComElec["PeakEnergy(w)"] / sf)

# get_peak_ComElec

In [65]:
# Get scenario and find the max
Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "1Eexist"]
Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "2Eclear"]
Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "3Elowe"]
NWexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "1NWexist"]
NWclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "2NWclear"]
NWlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "3NWlowe"]
SWexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "1SWexist"]
SWclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "2SWclear"]
SWlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "3SWlowe"]

Eexist


,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
0,1Eexist,01/01 01:00:00,E,Exist,0,1.691707e+06,1389807.55,0.0,39312,3.120826e+06,866.896199,4.128077
1,1Eexist,01/01 02:00:00,E,Exist,0,1.980665e+06,1389807.55,0.0,39312,3.409785e+06,947.162366,4.510297
2,1Eexist,01/01 03:00:00,E,Exist,0,2.001692e+06,1389807.55,0.0,16632,3.408132e+06,946.703261,4.508111
3,1Eexist,01/01 04:00:00,E,Exist,0,1.983465e+06,1389807.55,0.0,16632,3.389905e+06,941.640154,4.484001
4,1Eexist,01/01 05:00:00,E,Exist,0,1.958731e+06,1389807.55,0.0,16632,3.365171e+06,934.769730,4.451284
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1Eexist,12/31 20:00:00,E,Exist,0,5.180393e+05,1389807.55,0.0,151200,2.059047e+06,571.957451,2.723607
8756,1Eexist,12/31 21:00:00,E,Exist,0,3.950540e+05,1389807.55,0.0,151200,1.936062e+06,537.794883,2.560928
8757,1Eexist,12/31 22:00:00,E,Exist,0,4.359633e+05,1389807.55,0.0,151200,1.976971e+06,549.158568,2.615041
8758,1Eexist,12/31 23:00:00,E,Exist,0,7.373538e+05,1389807.55,0.0,128520,2.255681e+06,626.578139,2.983705


In [75]:
Eexist

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
0,1Eexist,01/01 01:00:00,E,Exist,0,1.691707e+06,1389807.55,0.0,39312,3.120826e+06,866.896199,4.128077
1,1Eexist,01/01 02:00:00,E,Exist,0,1.980665e+06,1389807.55,0.0,39312,3.409785e+06,947.162366,4.510297
2,1Eexist,01/01 03:00:00,E,Exist,0,2.001692e+06,1389807.55,0.0,16632,3.408132e+06,946.703261,4.508111
3,1Eexist,01/01 04:00:00,E,Exist,0,1.983465e+06,1389807.55,0.0,16632,3.389905e+06,941.640154,4.484001
4,1Eexist,01/01 05:00:00,E,Exist,0,1.958731e+06,1389807.55,0.0,16632,3.365171e+06,934.769730,4.451284
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1Eexist,12/31 20:00:00,E,Exist,0,5.180393e+05,1389807.55,0.0,151200,2.059047e+06,571.957451,2.723607
8756,1Eexist,12/31 21:00:00,E,Exist,0,3.950540e+05,1389807.55,0.0,151200,1.936062e+06,537.794883,2.560928
8757,1Eexist,12/31 22:00:00,E,Exist,0,4.359633e+05,1389807.55,0.0,151200,1.976971e+06,549.158568,2.615041
8758,1Eexist,12/31 23:00:00,E,Exist,0,7.373538e+05,1389807.55,0.0,128520,2.255681e+06,626.578139,2.983705


In [66]:
EexistPeak = (Eexist[Eexist["PeakEnergy[J](Hourly)"] == Eexist["PeakEnergy[J](Hourly)"].max()])
EclearPeak = (Eclear[Eclear["PeakEnergy[J](Hourly)"] == Eclear["PeakEnergy[J](Hourly)"].max()])
ElowePeak = (Elowe[Elowe["PeakEnergy[J](Hourly)"] == Elowe["PeakEnergy[J](Hourly)"].max()])
NWexistPeak = (NWexist[NWexist["PeakEnergy[J](Hourly)"] == NWexist["PeakEnergy[J](Hourly)"].max()])
NWclearPeak = (NWclear[NWclear["PeakEnergy[J](Hourly)"] == NWclear["PeakEnergy[J](Hourly)"].max()])
NWlowePeak = (NWlowe[NWlowe["PeakEnergy[J](Hourly)"] == NWlowe["PeakEnergy[J](Hourly)"].max()])
SWexistPeak = (SWexist[SWexist["PeakEnergy[J](Hourly)"] == SWexist["PeakEnergy[J](Hourly)"].max()])
SWclearPeak = (SWclear[SWclear["PeakEnergy[J](Hourly)"] == SWclear["PeakEnergy[J](Hourly)"].max()])
SWlowePeak = (SWlowe[SWlowe["PeakEnergy[J](Hourly)"] == SWlowe["PeakEnergy[J](Hourly)"].max()])


In [74]:
EexistPeak


,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
4457,1Eexist,07/05 18:00:00,E,Exist,0,0.0,1389807.55,6181707.401,42336,7613850.951,2114.958597,10.071231


In [67]:
PeakDemandElec = EexistPeak.append([EclearPeak,
ElowePeak,
NWexistPeak,
NWclearPeak,
NWlowePeak,
SWexistPeak,
SWclearPeak,
SWlowePeak
])

PeakDemandElec

,Scenario,Date_Time,Elevation,GlazingSystem,HeatingElecEnergy[J](Hourly),HeatingGasEnergy[J](Hourly),FanEnergy[J](Hourly),CoolingEnergy[J](Hourly),LightingEnergy[J](Hourly),PeakEnergy[J](Hourly),PeakEnergy(w),PeakEnergy(w/sf)
4457,1Eexist,07/05 18:00:00,E,Exist,0,0.000,1.389808e+06,6181707.401,42336,7.613851e+06,2114.958597,10.071231
30737,2Eclear,07/05 18:00:00,E,Clear Panel,0,0.000,1.246103e+06,5549782.498,42336,6.838222e+06,1899.506089,9.045267
57017,3Elowe,07/05 18:00:00,E,LowE Panel,0,0.000,1.082904e+06,4847378.509,42336,5.972619e+06,1659.060725,7.900289
18221,1NWexist,01/30 06:00:00,NW,Exist,0,8447341.134,6.662362e+05,0.000,33264,9.146841e+06,2540.789262,12.098996
44501,2NWclear,01/30 06:00:00,NW,Clear Panel,0,6978378.612,6.025776e+05,0.000,33264,7.614220e+06,2115.061178,10.071720
70781,3NWlowe,01/30 06:00:00,NW,LowE Panel,0,7272074.636,5.281277e+05,0.000,33264,7.833466e+06,2175.962877,10.361728
9461,1SWexist,01/30 06:00:00,SW,Exist,0,7765233.775,9.538411e+05,0.000,33264,8.752339e+06,2431.205230,11.577168
35741,2SWclear,01/30 06:00:00,SW,Clear Panel,0,6351347.720,8.402333e+05,0.000,33264,7.224845e+06,2006.901392,9.556673
62021,3SWlowe,01/30 06:00:00,SW,LowE Panel,0,6756342.809,7.402696e+05,0.000,33264,7.529876e+06,2091.632326,9.960154


In [ ]:
# # Export dataframe  to csv
# PeakDemandElec.to_csv("Scraper_Output/msp_peak_com.csv", header=True, index=False)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Export to CSV files


In [ ]:
# Define the scenario for the existing condition.
baselinePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Exist"]

# baselinePeakDemandElec

In [ ]:
# list(baselinePeakDemandElec.columns.values)

In [ ]:
# Rename columns to indicate baseline to prepare for merge"

allbaselinePeakDemandElec = baselinePeakDemandElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'B_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'B_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'B_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'B_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'B_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'B_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'B_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'B_PeakEnergy(w/sf)'
                            })

# allbaselinePeakDemandElec

In [ ]:
#Reset index
allbaselinePeakDemandElec.reset_index(inplace=True, drop=True)

# allbaselinePeakDemandElec

In [ ]:
# Define the scenario for the clear panel condition.
clearPeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Clear Panel"]

# clearPeakDemandElec

In [ ]:
# Rename columns to indicate baseline to prepare for merge"

allClearPeakDemandElec = clearPeakDemandElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'C_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'C_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'C_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'C_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'C_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'C_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'C_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'C_PeakEnergy(w/sf)'
                            })

# allClearPeakDemandElec

In [ ]:
#Reset index
allClearPeakDemandElec.reset_index(inplace=True, drop=True)

# allClearPeakDemandElec

In [ ]:
# Define the scenario for the lowe panel condition.
lowePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "LowE Panel"]

# lowePeakDemandElec

In [ ]:
# Rename columns to indicate baseline to prepare for merge"

allLowePeakDemandElec = lowePeakDemandElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'L_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'L_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'L_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'L_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'L_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'L_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'L_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'L_PeakEnergy(w/sf)'
                            })

# allLowePeakDemandElec

In [ ]:
# Reset index
allLowePeakDemandElec.reset_index(inplace=True, drop=True)

# allLowePeakDemandElec

---
## Merge the 3 dataframes to perform delta peak from existing calculation


In [ ]:
# merge existing and clear
MergedAnnualPeakData = pd.merge(allbaselinePeakDemandElec, allClearPeakDemandElec, left_index=True, right_index=True)

MergedAnnualPeakData

#merge the above with lowe
AllMergedAnnualPeakData = pd.merge(MergedAnnualPeakData, allLowePeakDemandElec, left_index=True, right_index=True)

# AllMergedAnnualPeakData

In [ ]:
# list(AllMergedAnnualPeakData.columns.values)

In [ ]:
#Deltas of exist - clear

AllMergedAnnualPeakData["DeltaClear_PeakEnergy(w)"] = AllMergedAnnualPeakData["B_PeakEnergy(w)"] - AllMergedAnnualPeakData["C_PeakEnergy(w)"]
AllMergedAnnualPeakData["DeltaClear_PeakEnergy(w/sf)"] = AllMergedAnnualPeakData["B_PeakEnergy(w/sf)"] - AllMergedAnnualPeakData["C_PeakEnergy(w/sf)"]

# AllMergedAnnualPeakData

In [ ]:
# Deltas of exist - lowe

AllMergedAnnualPeakData["DeltaLowe_PeakEnergy(w)"] = AllMergedAnnualPeakData["B_PeakEnergy(w)"] - AllMergedAnnualPeakData["L_PeakEnergy(w)"]
AllMergedAnnualPeakData["DeltaLowe_PeakEnergy(w/sf)"] = AllMergedAnnualPeakData["B_PeakEnergy(w/sf)"] - AllMergedAnnualPeakData["L_PeakEnergy(w/sf)"]

# AllMergedAnnualPeakData

In [ ]:
# list(AllMergedAnnualPeakData.columns.values)

In [ ]:
# Export grouped file to csv
AllMergedAnnualPeakData.to_csv("Scraper_Output/msp_deltas_peak_com.csv", header=True, index=False)